# Defining function to pull baseball data

* gives database name map with "name_map" dictionary

In [3]:
import requests 
from bs4 import BeautifulSoup 
from pprint import pprint
import pandas as pd
import numpy

def clean_MU(data, year, school):
    processed_data = []

    for row in range(0, len(data)-2):
        row_data = data[row].text.split()

        if len(data[row].text.split()) == 28:
            row_data = row_data[3:26]
            row_data[1:3] = [' '.join(row_data[1:3])]
            row_data[1] = row_data[1].replace('-', '.')
            row_data = [y for x in row_data for y in x.split('-')]
            row_data[1] = row_data[1].replace('.', '-')
            row_data.append(year)
            row_data.append(school)
            processed_data.append(
                dict(zip(db_names,row_data)))

        elif len(data[row].text.split()) == 30:
            row_data = row_data[4:28]
            row_data[1:4] = [' '.join(row_data[1:4])]
            row_data[1] = row_data[1].replace('-', '.')
            row_data = [y for x in row_data for y in x.split('-')]
            row_data[1] = row_data[1].replace('.', '-')
            row_data.append(year)
            processed_data.append(
                dict(zip(db_names,row_data)))

        else:
            print('Row with index of {} not stored'.format(row))
            print(data[row].text.split())
            
        
    global baseball_dict
    name = school+str(year)
    baseball_dict[name]= processed_data 
    
    


def pull_MU(site, year, school_code):
    url = site+str(year)
    r = requests.get(url)
    if r.status_code == 404:
        return
    soup = BeautifulSoup(r.content, "html")
    table = soup.table
    
    trs = table.find_all('tr')
    header_row = trs[0]
    
    names = []
    for column in header_row.find_all('th'):
        names.append(column.text)
    names.remove('Bio Link')
    
    global db_names
    table_data = []
    db_names = []
    for column in names:
        db_names.append(column.lower())
    table_data = table.find_all('tr')
    table_data = table_data[1:]
    
    global name_map
    names_full = ['Player Number', 'Name','Batting Average','On-base Plus Slugging',
             'Games Played','Games Started','At-bat','Run','Hit','Double','Triple','Home Run',
             'Runs Batted In','Total Bases','Slugging Percentage','Walk','Hit-by-pitch','Strikeout',
             'Ground Into Double Play','On-base Percentage','Sacrifice Fly','Sacrifice Bunt',
             'Stolen bases allowed','Stealing attempts', 'Year', 'School Code']
    db_names[4] = 'gp'
    db_names[21] = 'sb'
    db_names.insert(22,'att')
    db_names.insert(5, 'gs')
    db_names.append('year')
    db_names.append('school')
    name_map = dict(zip(db_names, names_full))
    
    clean_MU(table_data, year, school_code)

# Extracting and formatting all MU Baseball Data

In [ ]:
baseball_dict = {}
years = [x for x in range(1991,2020)]
sites = {'https://floridagators.com/sports/baseball/stats/': 'uf',
         'https://mutigers.com/sports/baseball/stats/': 'mu',
        'https://hailstate.com/sports/baseball/stats/': 'msu',
        'https://12thman.com/sports/baseball/stats/': 'a&m',
        'https://gamecocksonline.com/sports/baseball/stats/': 'usc',
        'https://olemisssports.com/sports/baseball/stats/': 'um',
        'https://auburntigers.com/sports/baseball/stats/': 'au',
        'https://georgiadogs.com/sports/baseball/stats/': 'uga',
        'https://ukathletics.com/sports/baseball/stats/': 'uk',
        'https://utsports.com/sports/baseball/stats/': 'utk',
        'https://rolltide.com/sports/baseball/stats': 'ua'}

for y in sites:
    for x in years:
        pull_MU(y, x, sites[y])


# Insert data into Pandas Dataframe

* Combines all df into one
* Saves as HDF5 file

In [6]:
def combine_all(dicto, division, first = True):
    for x in baseball_dict.keys():
        if first ==True:
            df = pd.DataFrame(dicto[x])
            first = False
        else:
            df1 = pd.DataFrame(dicto[x])
            df = pd.concat([df,df1])
            
    #changes string numbers to floats or int
    ints = [0,4,5,6,7,8,9,10,11,12,13,15,16,17,18,20,21,22,23]
    floats = [2, 3,14,19]
    try:
        for row in range(0, df.shape[0]):
            for col1 in ints:
                temp1 = df.iloc[row,col1]
                df.iloc[row,col1] = int(temp1)
            for col2 in floats:
                temp2 = df.iloc[row,col2]
                df.iloc[row,col2]=float(temp2)
    except:
        print('[{},{}]'.format(row,col1))
        print('error')
            
    filepath = "./dataframes/baseball.h5"
    df.to_hdf(filepath, key = division)            

In [7]:
combine_all(baseball_dict, 'sec')

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['player', 'avg', 'ops', 'slg%', 'ob%', 'school']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


# Opening saved file and queries to make sure it read correctly

In [9]:
reread = pd.read_hdf('./dataframes/baseball.h5', key = 'sec')

In [21]:
reread.loc[reread['year'] == 1994]

,#,player,avg,ops,gp,gs,ab,r,h,2b,...,hbp,so,gdp,ob%,sf,sh,sb,att,year,school
0,20,"Waggoner, Jay",0.409,1.018,65,65,291,69,119,23,...,1,22,5,0.465,2,4,11,14,1994,au
1,4,"Moore, Brandon",0.361,0.896,65,65,269,56,97,21,...,3,21,6,0.405,6,3,6,7,1994,au
2,15,"Killimett, Mike",0.344,0.884,64,61,259,48,89,19,...,1,33,3,0.382,5,3,2,6,1994,au
3,5,"Whittenburg, Russell",0.307,0.736,53,31,127,19,39,6,...,3,22,0,0.382,1,4,3,4,1994,au
4,1,"Key, Ken",0.305,0.758,53,40,164,26,50,5,...,3,22,0,0.374,0,7,4,5,1994,au
5,24,"Chabot, Kevin",0.296,0.941,58,52,196,44,58,8,...,6,61,1,0.421,1,0,3,4,1994,au
6,25,"McNally, Shawn",0.271,0.717,65,65,251,62,68,9,...,5,37,2,0.37,4,10,12,17,1994,au
7,11,"Lewis, Robert",0.256,0.762,64,59,176,38,45,8,...,20,41,1,0.404,2,7,0,2,1994,au
8,23,"Sanders, Frank",0.249,0.7,61,51,185,44,46,6,...,2,58,1,0.365,0,5,9,11,1994,au
9,8,"Bellhorn, Mark",0.377,1.106,40,40,151,54,57,14,...,2,24,4,0.51,2,1,12,13,1994,au
